<a href="https://colab.research.google.com/github/dcolinmorgan/test/blob/master/LTCOPD_TF_pheno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
!pip install GEOparse
import GEOparse
gse = GEOparse.get_GEO(geo="GSE76925", destdir="./")

In [4]:
gpl = gse.gpls[next(iter(gse.gpls))]
data = gse.pivot_samples(values="VALUE")#, gpl=gpl, annotation_column="Symbol")
data.sort_index
pheno=gse.phenotype_data.rename(columns={'characteristics_ch1.0.age': 'age',"characteristics_ch1.1.Sex":"sex","characteristics_ch1.3.bmi":"bmi","characteristics_ch1.6.fev1.pp":"fev1","characteristics_ch1.4.packyears":"PY"})
# pheno[['age',"sex","bmi","fev1","PY"]]#.to_csv('pheno_data.txt',sep='\t',index=True,header=True)

In [7]:
pheno=gse.phenotype_data.rename(columns={'characteristics_ch1.0.age': 'age',"characteristics_ch1.1.Sex":"sex",
                                         'characteristics_ch1.2.race': 'race',"characteristics_ch1.3.bmi":"bmi",
                                         "characteristics_ch1.6.fev1.pp":"fev1","characteristics_ch1.4.packyears":"PY",
                                         'characteristics_ch1.7.fev1fvc':'fev1fvc','characteristics_ch1.8.laa950':'laa950',
                                         'characteristics_ch1.9.perc15':'perc15','characteristics_ch1.10.pi10':'pi10',
                                         'characteristics_ch1.5.copd':'copd'})
pheno=pheno[['age',"sex","bmi","fev1","PY","race",'fev1fvc','laa950','perc15','pi10','copd']]

In [8]:
dataframe=pheno
dataframe.replace('NA','nan',inplace=True)
dataframe=pd.DataFrame(dataframe)
dataframe.age=dataframe.age.astype('float')
dataframe.fev1=dataframe.fev1.astype('float')
dataframe.PY=dataframe.PY.astype('float')
dataframe.fev1fvc=dataframe.fev1fvc.astype('float')

dataframe.bmi=dataframe.bmi.fillna(0)
dataframe.laa950=dataframe.laa950.fillna(0)
dataframe.perc15=dataframe.perc15.fillna(0)
dataframe.pi10=dataframe.pi10.fillna(0)

dataframe.bmi=dataframe.bmi.astype('float')
dataframe.laa950=dataframe.laa950.astype('float')
dataframe.perc15=dataframe.perc15.astype('float')
dataframe.pi10=dataframe.pi10.astype('float')

dataframe.PY=dataframe.PY.astype('int')

dataframe.dtypes

age        float64
sex         object
bmi        float64
fev1       float64
PY           int64
race        object
fev1fvc    float64
laa950     float64
perc15     float64
pi10       float64
copd        object
dtype: object

In [9]:
# In the original dataset "4" indicates the pet was not adopted.
dataframe['target'] = np.where(dataframe['copd']=='cont', 0, 1)

# Drop un-used columns.
# dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])

## Split the dataframe into train, validation, and test

The dataset we downloaded was a single CSV file. We will split this into train, validation, and test sets.

In [10]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

96 train examples
24 validation examples
31 test examples


## Create an input pipeline using tf.data

Next, we will wrap the dataframes with [tf.data](https://www.tensorflow.org/guide/datasets). This will enable us  to use feature columns as a bridge to map from the columns in the Pandas dataframe to features used to train the model. If we were working with a very large CSV file (so large that it does not fit into memory), we would use tf.data to read it from disk directly. That is not covered in this tutorial.

In [11]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [12]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

## Understand the input pipeline

Now that we have created the input pipeline, let's call it to see the format of the data it returns. We have used a small batch size to keep the output readable.

In [13]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['age'])
  print('A batch of targets:', label_batch )

Every feature: ['age', 'sex', 'bmi', 'fev1', 'PY', 'race', 'fev1fvc', 'laa950', 'perc15', 'pi10', 'copd']
A batch of ages: tf.Tensor([55. 74. 65. 48. 63.], shape=(5,), dtype=float64)
A batch of targets: tf.Tensor([0 1 1 1 1], shape=(5,), dtype=int64)


We can see that the dataset returns a dictionary of column names (from the dataframe) that map to column values from rows in the dataframe.

## Demonstrate several types of feature columns
TensorFlow provides many types of feature columns. In this section, we will create several types of feature columns, and demonstrate how they transform a column from the dataframe.

In [14]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

In [15]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())


### Numeric columns
The output of a feature column becomes the input to the model (using the demo function defined above, we will be able to see exactly how each column from the dataframe is transformed). A [numeric column](https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column) is the simplest type of column. It is used to represent real valued features. When using this column, your model will receive the column value from the dataframe unchanged.

In [16]:
# dataframe.PY=dataframe.PY.astype('float')

photo_count = feature_column.numeric_column('PY')
demo(photo_count)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[ 40.]
 [120.]
 [ 70.]
 [ 45.]
 [ 11.]]


In the PetFinder dataset, most columns from the dataframe are categorical.

### Bucketized columns
Often, you don't want to feed a number directly into the model, but instead split its value into different categories based on numerical ranges. Consider raw data that represents a person's age. Instead of representing age as a numeric column, we could split the age into several buckets using a [bucketized column](https://www.tensorflow.org/api_docs/python/tf/feature_column/bucketized_column). Notice the one-hot values below describe which age range each row matches.

In [17]:
dataframe.replace('NA','nan',inplace=True)
dataframe.bmi.fillna(0).astype('float').describe()

count    151.000000
mean      25.929205
std        5.862117
min        0.000000
25%       22.505000
50%       25.890000
75%       29.840000
max       51.350000
Name: bmi, dtype: float64

In [18]:
age = feature_column.numeric_column('age')

age_buckets = feature_column.bucketized_column(age, boundaries=[42, 60, 63,67,86])
demo(age_buckets)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


### Categorical columns
In this dataset, Type is represented as a string (e.g. 'Dog', or 'Cat'). We cannot feed strings directly to a model. Instead, we must first map them to numeric values. The categorical vocabulary columns provide a way to represent strings as a one-hot vector (much like you have seen above with age buckets). The vocabulary can be passed as a list using [categorical_column_with_vocabulary_list](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_list), or loaded from a file using [categorical_column_with_vocabulary_file](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_file).

In [19]:
animal_type = feature_column.categorical_column_with_vocabulary_list(
      'sex', ['M', 'F'])

animal_type_one_hot = feature_column.indicator_column(animal_type)
demo(animal_type_one_hot)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]


### Embedding columns
Suppose instead of having just a few possible strings, we have thousands (or more) values per category. For a number of reasons, as the number of categories grow large, it becomes infeasible to train a neural network using one-hot encodings. We can use an embedding column to overcome this limitation. Instead of representing the data as a one-hot vector of many dimensions, an [embedding column](https://www.tensorflow.org/api_docs/python/tf/feature_column/embedding_column) represents that data as a lower-dimensional, dense vector in which each cell can contain any number, not just 0 or 1. The size of the embedding (8, in the example below) is a parameter that must be tuned.

Key point: using an embedding column is best when a categorical column has many possible values. We are using one here for demonstration purposes, so you have a complete example you can modify for a different dataset in the future.

In [20]:
# Notice the input to the embedding column is the categorical column
# we previously created
breed1 = feature_column.categorical_column_with_vocabulary_list(
      'race', dataframe.race.unique())
breed1_embedding = feature_column.embedding_column(breed1, dimension=8)
demo(breed1_embedding)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[ 0.4204154  -0.42070276 -0.40110764  0.28362778 -0.5102332   0.17261975
   0.0629276   0.15593086]
 [ 0.4204154  -0.42070276 -0.40110764  0.28362778 -0.5102332   0.17261975
   0.0629276   0.15593086]
 [ 0.4204154  -0.42070276 -0.40110764  0.28362778 -0.5102332   0.17261975
   0.0629276   0.15593086]
 [ 0.4204154  -0.42070276 -0.40110764  0.28362778 -0.5102332   0.17261975
   0.0629276   0.15593086]
 [ 0.4204154  -0.42070276 -0.40110764  0.28362778 -0.5102332   0.17261975
   0.0629276   0.15593086]]


### Hashed feature columns

Another way to represent a categorical column with a large number of values is to use a [categorical_column_with_hash_bucket](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_hash_bucket). This feature column calculates a hash value of the input, then selects one of the `hash_bucket_size` buckets to encode a string. When using this column, you do not need to provide the vocabulary, and you can choose to make the number of hash_buckets significantly smaller than the number of actual categories to save space.

Key point: An important downside of this technique is that there may be collisions in which different strings are mapped to the same bucket. In practice, this can work well for some datasets regardless.

In [21]:
breed1_hashed = feature_column.categorical_column_with_hash_bucket(
      'race', hash_bucket_size=10)
demo(feature_column.indicator_column(breed1_hashed))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]


### Crossed feature columns
Combining features into a single feature, better known as [feature crosses](https://developers.google.com/machine-learning/glossary/#feature_cross), enables a model to learn separate weights for each combination of features. Here, we will create a new feature that is the cross of Age and Type. Note that `crossed_column` does not build the full table of all possible combinations (which could be very large). Instead, it is backed by a `hashed_column`, so you can choose how large the table is.

In [22]:
crossed_feature = feature_column.crossed_column([age_buckets, animal_type], hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


## Choose which columns to use
We have seen how to use several types of feature columns. Now we will use them to train a model. The goal of this tutorial is to show you the complete code (e.g. mechanics) needed to work with feature columns. We have selected a few columns to train our model below arbitrarily.

Key point: If your aim is to build an accurate model, try a larger dataset of your own, and think carefully about which features are the most meaningful to include, and how they should be represented.

In [23]:
feature_columns = []

# numeric cols
for header in ['age','bmi','PY','fev1','fev1fvc','laa950','perc15','pi10']:
  feature_columns.append(feature_column.numeric_column(header))

In [24]:
# bucketized cols
age = feature_column.numeric_column('age')
age_buckets = feature_column.bucketized_column(age, boundaries=[42, 60, 63,67,86])
feature_columns.append(age_buckets)

In [25]:
dataframe.age=dataframe.age.astype('int')
dataframe.fev1=dataframe.fev1.astype('int')
dataframe.PY=dataframe.PY.astype('int')


In [27]:
# indicator_columns
indicator_column_names = ['copd']

for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, dataframe[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [28]:
# embedding columns
breed1 = feature_column.categorical_column_with_vocabulary_list(
      'race', dataframe.race.unique())
breed1_embedding = feature_column.embedding_column(breed1, dimension=8)
feature_columns.append(breed1_embedding)

In [29]:
# crossed columns
age_type_feature = feature_column.crossed_column([age_buckets, animal_type], hash_bucket_size=100)
feature_columns.append(feature_column.indicator_column(age_type_feature))

### Create a feature layer
Now that we have defined our feature columns, we will use a [DenseFeatures](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/DenseFeatures) layer to input them to our Keras model.

In [30]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

Earlier, we used a small batch size to demonstrate how feature columns worked. We create a new input pipeline with a larger batch size.

In [31]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)


## Create, compile, and train the model

In [32]:
tf.keras.backend.set_floatx('float32')
# tf.keras.backend.set_int('int32')

model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
Consider rewriting this model with the Functional API.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Consider rewriting this model with the Functional API.
1/3 [=========>....................] - ETA: 0s - loss: 8.4556 - accuracy: 0.3438WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'age': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=float64>, 'sex': <tf.Tensor 'ExpandDims_10:0' shape=(None, 1) dtype=string>, 'bmi': <tf.Tensor 'ExpandDims_2:0' shape=(Non

In [33]:
val_ds

<BatchDataset shapes: ({age: (None,), sex: (None,), bmi: (None,), fev1: (None,), PY: (None,), race: (None,), fev1fvc: (None,), laa950: (None,), perc15: (None,), pi10: (None,), copd: (None,)}, (None,)), types: ({age: tf.float64, sex: tf.string, bmi: tf.float64, fev1: tf.float64, PY: tf.int64, race: tf.string, fev1fvc: tf.float64, laa950: tf.float64, perc15: tf.float64, pi10: tf.float64, copd: tf.string}, tf.int64)>

In [34]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1/1 [==============================] - 0s 1ms/step - loss: 0.6627 - accuracy: 0.2581
Accuracy 0.25806450843811035


Key point: You will typically see best results with deep learning with much larger and more complex datasets. When working with a small dataset like this one, we recommend using a decision tree or random forest as a strong baseline. The goal of this tutorial is not to train an accurate model, but to demonstrate the mechanics of working with structured data, so you have code to use as a starting point when working with your own datasets in the future.

## Next steps
The best way to learn more about classifying structured data is to try it yourself. We suggest finding another dataset to work with, and training a model to classify it using code similar to the above. To improve accuracy, think carefully about which features to include in your model, and how they should be represented.